In [49]:
import os
from pytse_client.download import download_financial_indexes
import pandas as pd
import jalali_pandas
import math
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz 
import warnings
warnings.filterwarnings("ignore")

desktop = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop') 
bourse_view = os.path.join(desktop, "sector_slc", "bourse_view")
dataset_path = os.path.join(bourse_view, "dataset")
dataset_df = pd.read_csv(dataset_path + "\\" + "dataset.csv")
dataset_df = dataset_df.drop("Unnamed: 0",axis=1)
list_of_sectors = dataset_df.sector.unique().tolist()
dataset_df["P/E-ttm_1_lag"] = dataset_df["P/E-ttm_1_lag"] - dataset_df["market_pe_1_lag"] 
dataset_df["P/E-ttm_2_lag"] = dataset_df["P/E-ttm_2_lag"] - dataset_df["market_pe_2_lag"] 
dataset_df["quarterly_return_1_lag_excess_usd"] = dataset_df["quarterly_return_1_lag"] - dataset_df["quarterly_usd_return_1_lag"]
dataset_df["quarterly_return_2_lag_excess_usd"] = dataset_df["quarterly_return_2_lag"] - dataset_df["quarterly_usd_return_2_lag"]
dataset_df["quarterly_return_1_lag_excess_index"] = dataset_df["quarterly_return_1_lag"] - dataset_df["index_quarterly_return_1_lag"]
dataset_df["quarterly_return_2_lag_excess_index"] = dataset_df["quarterly_return_2_lag"] - dataset_df["index_quarterly_return_2_lag"]


In [48]:
year_to_predict = 1396
quarter_to_predict = 1
first_occurence = dataset_df[(dataset_df.quarter == quarter_to_predict) & (dataset_df.year == year_to_predict)].iloc[0].name
train_set = dataset_df[:first_occurence]

list_of_features = ["P/E-ttm_1_lag", "P/E-ttm_2_lag", "quarterly_return_1_lag", "quarterly_return_2_lag",
                    "quarterly_return_1_lag_excess_usd", "quarterly_return_2_lag_excess_usd",
                    "quarterly_return_1_lag_excess_index", "quarterly_return_2_lag_excess_index"
                    ]
target = "quarter_excess_return"
regressor = DecisionTreeRegressor(random_state = 0)
X = train_set[list_of_features]
y = train_set[target]
regressor.fit(X, y)

result_df = pd.DataFrame(columns=["sector", "predicted_excess_return","realized_excess_return"])
for sector in list_of_sectors:
    row = dataset_df[(dataset_df.quarter == quarter_to_predict) & (dataset_df.year == year_to_predict) & (dataset_df.sector == sector)].iloc[0]
    row_input = row[list_of_features]
    predicted_excess_return = regressor.predict([row_input])
    realized_excess_return = row["quarter_excess_return"] 
    new_row = [sector, predicted_excess_return[0]*100, realized_excess_return*100]
    result_df.loc[len(result_df)] = new_row
result_df = result_df.sort_values(by="predicted_excess_return",ascending=False).reset_index().drop("index",axis=1)
long_leg_return = result_df.realized_excess_return.iloc[:5].mean()
short_leg_return = result_df.realized_excess_return.iloc[-5:].mean()
index_return = row.index_quarterly_return * 100
print(long_leg_return,short_leg_return,index_return)
result_df

11.242009025725208 5.207196044991462 2.1417951431212003


,sector,predicted_excess_return,realized_excess_return
0,حمل و نقل بار زمینی,48.675348,7.666307
1,محصولات کاغذی,36.635792,27.853017
2,نوشیدنی,31.125327,0.420438
3,چوب,31.125327,-4.449728
4,تجهیزات صنعتی,28.601467,24.720010
5,شکر,19.785958,43.709492
6,آهن و فولاد,16.741690,-5.076896
7,تولید کود و ترکیبات نیتروژن,9.653093,-13.979326
8,مواد شیمیایی-متنوع,9.653093,2.035065
9,نرم افزار و خدمات,9.653093,37.795463
